In [1]:
!pip install -q streamlit pyngrok scikit-learn pandas matplotlib seaborn shap tensorflow


from pyngrok import ngrok
ngrok.set_auth_token("2zywGxhmq42Np0vrLeN9xDCF2sA_MXPqhnVjgFqNuW5Qwx1B")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.0 MB/s eta 0:00:00


In [2]:
app_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model

# Load model and scaler
model = load_model('heart_nn_model.keras')
scaler = joblib.load('scaler.save')

st.title("Heart Disease Prediction App")

# Input fields
age = st.slider("Age", 20, 80, 50)
sex = st.radio("Sex", ["Male", "Female"])
cp = st.selectbox("Chest Pain Type (cp)", [0, 1, 2, 3])
trestbps = st.slider("Resting Blood Pressure (trestbps)", 80, 200, 120)
chol = st.slider("Serum Cholesterol (chol)", 100, 600, 240)
fbs = st.radio("Fasting Blood Sugar > 120 mg/dl (fbs)", [0, 1])
restecg = st.selectbox("Resting ECG results (restecg)", [0, 1, 2])
thalach = st.slider("Max Heart Rate (thalach)", 60, 210, 150)
exang = st.radio("Exercise Induced Angina (exang)", [0, 1])
oldpeak = st.slider("ST Depression (oldpeak)", 0.0, 6.0, 1.0)
slope = st.selectbox("Slope of Peak Exercise ST segment (slope)", [0, 1, 2])
ca = st.selectbox("Number of major vessels colored (ca)", [0, 1, 2, 3])
thal = st.selectbox("Thalassemia (thal)", [0, 1, 2])

# Encoding sex
sex = 1 if sex == "Male" else 0

# Prediction
if st.button("Predict"):
    input_data = np.array([[age, sex, cp, trestbps, chol, fbs, restecg,
                            thalach, exang, oldpeak, slope, ca, thal]])
    input_scaled = scaler.transform(input_data)
    pred = model.predict(input_scaled)[0][0]
    result = "🔴 Heart Disease Likely" if pred > 0.5 else "🟢 No Heart Disease"
    st.subheader(f"Prediction: {result}")
    st.caption(f"Probability: {pred:.2f}")
'''

In [3]:
with open('app.py', 'w') as f:
    f.write(app_code)

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib

In [6]:
df = pd.read_csv('/content/heart_cleveland_upload (1) (1).csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [7]:
if 'target' not in df.columns and 'condition' in df.columns:
    df['target'] = df['condition'].apply(lambda x: 1 if x > 0 else 0)
    df.drop('condition', axis=1, inplace=True)

X = df.drop('target', axis=1)
y = df['target']

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, 'scaler.save')

['scaler.save']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
model = Sequential([
    Dense(16, input_dim=X_train.shape[1], activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, verbose=0)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.save("heart_nn_model.keras")

In [12]:
!pkill streamlit || true
!streamlit run app.py &>/content/log.txt &

In [13]:
public_url = ngrok.connect(8501)
print("🔗 Click this link to open the app:", public_url)

🔗 Click this link to open the app: NgrokTunnel: "https://e3ff917e60b3.ngrok-free.app" -> "http://localhost:8501"
